In [ ]:
import tensorflow as tf
physical_device=tf.config.experimental.list_physical_devices('GPU')
print(len(physical_device))
tf.config.experimental.set_memory_growth(physical_device[0],True)
!pip install -q kaggle
from google.colab import  files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d paultimothymooney/recipenlg
!unzip recipenlg.zip
!rm -rf /content/RecipeNLG_code /content/recipenlg.zip /content/RecipeNLG_license.png /content/RecipeNLG_paper.pdf


1


Saving kaggle.json to kaggle.json
 98% 632M/643M [00:06<00:00, 188MB/s]
100% 643M/643M [00:06<00:00, 108MB/s]
Archive:  recipenlg.zip
  inflating: RecipeNLG_code/README.md  
  inflating: RecipeNLG_code/eval/evaluation.ipynb  
  inflating: RecipeNLG_code/generation/README.md  
  inflating: RecipeNLG_code/generation/SAMPLE.md  
  inflating: RecipeNLG_code/generation/dataset2text.ipynb  
  inflating: RecipeNLG_code/generation/preparation.py  
  inflating: RecipeNLG_code/generation/run_generation.py  
  inflating: RecipeNLG_code/generation/run_lm_finetuning.py  
  inflating: RecipeNLG_code/generation/run_lm_finetuning_new.py  
  inflating: RecipeNLG_code/generation/run_lm_finetuning_tpu.py  
  inflating: RecipeNLG_code/generation/tokenization.py  
  inflating: RecipeNLG_code/ner/Language2_0.ipynb  
  inflating: RecipeNLG_code/ner/NER.ipynb  
  inflating: RecipeNLG_code/ner/model/meta.json  
  inflating: RecipeNLG_code/ner/model/ner/cfg  
  inflating: RecipeNLG_code/ner/model/ner/model  
  

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_csv("/content/RecipeNLG_dataset.csv")[0:1050]
df=df.drop(columns=['Unnamed: 0','link','source'])

In [ ]:
df

,title,ingredients,directions,NER
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...","[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....","[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...","[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...","[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...","[""peanut butter"", ""graham cracker crumbs"", ""bu..."
...,...,...,...,...
1045,Corn Casserole,"[""1 can whole corn"", ""1 can creamed corn"", ""1 ...","[""Melt margarine."", ""Mix remaining ingredients...","[""corn"", ""corn"", ""corn mix"", ""sour cream"", ""ma..."
1046,Honey'S Hominy Soup,"[""1 medium onion, chopped"", ""1 green pepper, c...","[""Saute peppers and onion in butter until tran...","[""onion"", ""green pepper"", ""butter"", ""chicken b..."
1047,Stuffed Cabbage,"[""1 medium head cabbage"", ""1 onion"", ""1/2 tsp....","[""Boil cabbage for 30 minutes. Drain and cool....","[""head cabbage"", ""onion"", ""garlic powder"", ""sa..."
1048,Pizza Dip,"[""1 lb. ground beef"", ""1/4 c. minced onion"", ""...","[""Saute ground beef with onion and garlic in s...","[""ground beef"", ""onion"", ""garlic"", ""tomato sau..."


In [ ]:
temp=df.copy()
import ast
allIngredients=[]
for i in range(temp.shape[0]):
    x=ast.literal_eval(temp['NER'][i])
    for j in range(len(x)):
        allIngredients.append(x[j].lower())

In [ ]:
temp[allIngredients]=0

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3645: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = value


In [ ]:
for i in range(temp.shape[0]):
    x=ast.literal_eval(temp['NER'][i])
    for j in range(len(x)):
        temp.iloc[i,temp.columns.get_loc(x[j].lower())]=1

In [ ]:
import re
mylist = temp.columns.tolist()
r = re.compile(".*bacon.*|.*pork.*|lean")
newlist = list(filter(r.match, mylist)) # Read Note
print(newlist)

In [ ]:
newlist=['pork', 'pork roast', 'pork tenderloin', 'lean pork', 'pork sausage', 'ground pork', 'salt pork', 'pork chops', 'ground pork sausage', 'thin sliced pork chops', 'pork shoulder roast', 'pork shoulder']
x=temp[
(temp["title"] == "Pigs In Blanket")|
(temp["bacon"] == 1) | 
(temp["pork"] == 1) | 
(temp["bacon drippings"] == 1) | 
(temp["lean hamburger"] == 1) | 
(temp["hot bacon"] == 1) | 
(temp["bacon fat"] == 1) | 
(temp["pork roast"] == 1) | 
(temp["pork tenderloin"] == 1) | 
(temp["fully cooked canadian bacon"] == 1) | 
(temp["crisp fried bacon"] == 1) | 
(temp["lean pork"] == 1) | 
(temp["bacon bits"] == 1) | 
(temp["pork sausage"] == 1) | 
(temp["ground pork"] == 1) | 
(temp["salt pork"] == 1) | 
(temp["bacon grease"] == 1) | 
(temp["pork chops"] == 1) | 
(temp["lean"] == 1) | 
(temp["lean hamburg"] == 1) | 
(temp["ground pork sausage"] == 1) 
]
dropper=np.array(x["title"].index).tolist()
dropper
temp=temp.drop(newlist,axis=1)

In [ ]:
df.drop(dropper, axis=0, inplace=True)

In [ ]:
df=df.reset_index()

In [ ]:
df=df.drop("index",axis=1)

In [ ]:
temp=df.copy()
import ast
allIngredients=[]
for i in range(temp.shape[0]):
    x=ast.literal_eval(temp['NER'][i])
    for j in range(len(x)):
        allIngredients.append(x[j].lower())
temp[allIngredients]=0
for i in range(temp.shape[0]):
    x=ast.literal_eval(temp['NER'][i])
    for j in range(len(x)):
        temp.iloc[i,temp.columns.get_loc(x[j].lower())]=1

,title,ingredients,directions,NER,brown sugar,milk,vanilla,nuts,butter,bite size shredded rice biscuits,...,regular rice,minor beef base,stout,english cheese spread,level,fresh nonfat milk,carnation milk,corn mix,hominy,head cabbage
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...","[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu...",1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....","[""beef"", ""chicken breasts"", ""cream of mushroom...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...","[""frozen corn"", ""cream cheese"", ""butter"", ""gar...",0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...","[""chicken"", ""chicken gravy"", ""cream of mushroo...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...","[""peanut butter"", ""graham cracker crumbs"", ""bu...",0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045,Corn Casserole,"[""1 can whole corn"", ""1 can creamed corn"", ""1 ...","[""Melt margarine."", ""Mix remaining ingredients...","[""corn"", ""corn"", ""corn mix"", ""sour cream"", ""ma...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1046,Honey'S Hominy Soup,"[""1 medium onion, chopped"", ""1 green pepper, c...","[""Saute peppers and onion in butter until tran...","[""onion"", ""green pepper"", ""butter"", ""chicken b...",0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1047,Stuffed Cabbage,"[""1 medium head cabbage"", ""1 onion"", ""1/2 tsp....","[""Boil cabbage for 30 minutes. Drain and cool....","[""head cabbage"", ""onion"", ""garlic powder"", ""sa...",0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1048,Pizza Dip,"[""1 lb. ground beef"", ""1/4 c. minced onion"", ""...","[""Saute ground beef with onion and garlic in s...","[""ground beef"", ""onion"", ""garlic"", ""tomato sau...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
temp.to_csv("Cleaned_Data.csv")

In [ ]:
temp.shape

(995, 1096)

In [ ]:
import pandas as pd
import numpy as np
temp=pd.read_csv("/content/Cleaned_Data.csv")

In [ ]:
temp= pd.read_csv("/content/Cleaned_Data.csv")
allIngredients=temp.columns.tolist()
allIngredients.pop(0)
allIngredients.pop(0)
allIngredients.pop(0)
allIngredients.pop(0)
allIngredients.pop(0)
X=np.array(temp[allIngredients])

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
model=NearestNeighbors(n_neighbors=temp.shape[0], radius=0.2)
model.fit(X)


NearestNeighbors(n_neighbors=995, radius=0.2)

In [ ]:
import pickle
pickle_out=open("NearestNeighborModel.pkl","wb")
pickle.dump(model,pickle_out)
pickle_out.close()